# 若有在 Jetson Nano 上安裝 Tensorflow 可以直接運行

## get a pretrained mobilenet model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
model = MobileNet(weights='imagenet',
                  include_top=True)
# save to saved model
tf.saved_model.save(model, 'mobilenet_saved_model')

## tensorflow to trt

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
saved_file = 'mobilenet_saved_model'
TRT_file = 'mobilenet_saved_model_TFTRT_FP16'
print('Converting to TF-TRT FP16...')
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(
    max_workspace_size_bytes=(1<<28),
    precision_mode="FP16",
    maximum_cached_engines=1)
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_file,
    conversion_params=conversion_params)
converter.convert()
converter.save(output_saved_model_dir=TRT_file)
print('Done Converting to TF-TRT FP16')

## Inference

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions

x = tf.random.uniform([1, 224, 224, 3])

input_saved_model = 'mobilenet_saved_model_TFTRT_FP16'
saved_model_loaded = tf.saved_model.load(
    input_saved_model,
    tags=[tag_constants.SERVING])
signature_keys = list(saved_model_loaded.signatures.keys())
print(signature_keys)

infer = saved_model_loaded.signatures['serving_default']
print(infer.structured_outputs)
keys = list(infer.structured_outputs.keys())

preds = infer(x)[keys[0]].numpy()
print('Predicted: {}'.format(decode_predictions(preds, top=3)[0]))